# 가설
 - 기존엔 벡터검색만 수행했음
 - 이 경우엔 단순 유사성만 검색하기 때문에, 의미는 전혀 다르지만 비슷한 오류 양상 체크는 불가한 단점이 존재.
 - BM25도 조합하여 각각 or 하이브리드 방식으로 말뭉치 내 입력 문장과 유사한 오류 문장을 검색하는 방식으로 변경

## TODO: 추후 RAG 외 MCP도 응용해 볼 생각
 - 예를 들면, LLM이 직접 문장을 인식하고 나서, LLM이 오류인지 판단하게 한 후 이와 비슷한 양상을 국가 등을 필터한 후 말뭉치에서 능동적으로 검색

# 생각
 - 말뭉치에서 에러 문장 / 올바른 문장 / 에러 형태소 배열 / 올바른 형태소 배열을 추출
 - 사용자가 문장 입력 시, 형태소 분석하여 위 4개 모두 검색
   - 형태소 배열은 벡터 검색 불필요. BM25 사용
 - 형태소가 올바르지만, 잘못된 단어 사용 (예 - 결혼 / 계론) 과 같은 경우는?
   - (확인해봐야 할 거 같은데) 형태소는 OK, 벡터비교는 낮게 나올거 같음.
   - 그리고, 말뭉치에서 오타로 인한 오류라는 태그가 달려 있을거임. 이 오타는 어떻게 "검출" 하고, 동일한 사례를 어떻게 검색하나? LLM이 판단할 수 있는가?
     - (만약 오타를 비슷하게 낸다면) 이런 케이스들은 벡터검색 일치가 높게 나올 거 같음. 그리고, 이 경우엔 반드시 **오타 오류** 를 포함하고 있어야 함.
     - 대상이 초급이면, **오타 오류** 를 포함한 말뭉치에서 1차 검색하는게 좋을 수도 있음.
   - **오타 오류** / **문법 오류** 두가지 케이스로 나눌 수 있나? -> **이 부분 논의 필요**
     - 답변: 둘 다 많음. (**오타 오류** / **문법 오류** / **둘 다**)
   - 형태소 기반으로 검색을 할때엔 레벤슈타인 알고리즘 사용 (rapidfuzz)
     - 엘라스틱서치에서 관련 기능 지원 안해서, 1차로 검색한 후 2차 내에서 검색 보완하는 식으로 사용해야 할 듯

# 검색 순서
  1. 입력 문장을 형태소 분석해서 "입력 문장", "형태소 배열" 만듬. (필요시 출신국가 등 파라미터 추가)
  2. 입력 문장을 "그대로" 말뭉치 내 모든 문장에 대해 검색.
  3. 벡터 검색 결과 / BM25 검색 결과를 각각 별도로 추출함.
  4. 벡터/BM25 결과 일치수치가 100이면 해당 문장과 동일함. 이 경우 올바른 문장이던 틀린 문장이던 말뭉치에 에러정보가 존재함.
  5. 공통분모가 존재하고 벡터 < BM25가 유사성이 더 높으면 오타 오류일 확률이 높음. (특히 오타 오류가 끼어있으면)
  7. 공통분모가 존재하거 벡터 > BM25가 유사성이 더 높으면 "의미적으로만" 비슷할 확률이 높음.

# DB 구조
 - 에러 문장
 - 에러 문장 벡터
 - 틀린 형태소 배열
 - 올바른 형태소 배열


In [1]:
# 에러 문장, 에러 문장 벡터, 틀린 형태소 배열만 OpenSearch에 저장한 예제

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from opensearchpy import OpenSearch

/home/puju/py_playground/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# OpenSearch 클라이언트 연결
client = OpenSearch(
    hosts=[{'host': '172.30.1.81', 'port': 9200}],
    http_auth=None,
    use_ssl=False,
    verify_certs=False
)

In [3]:
# embedding 모델 불러오기
model = SentenceTransformer(
        "./model/KURE-v1",
        local_files_only=True,
)

Loading weights: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 391/391 [00:01<00:00, 288.45it/s, Materializing param=pooler.dense.weight]


In [5]:
# 인덱스 생성 (한국어 분석기 + 벡터 필드)
index_name = 'korean_test'

index_body = {
    "settings": {
        "index.knn": True,
        "analysis": {
            "tokenizer": {
                "nori_tokenizer": {
                    "type": "nori_tokenizer",
                    "decompound_mode": "mixed"
                }
            },
            "analyzer": {
                "korean": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "original_text": { # 원본 문자열
                "type": "text",
                "analyzer": "korean"
            },
            "morphs": { # 형태소 배열
                 "type": "keyword"
            },
            "embedding": { # 임베딩
                "type": "knn_vector",
                "dimension": 1024,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                    "engine": "lucene"  # nmslib → lucene으로 변경
                }
            }
        }
    }
}

# 기존 인덱스 삭제 (테스트용)
if client.indices.exists(index=index_name):
    client.indices.delete(index=index_name)

# 인덱스 생성
client.indices.create(index=index_name, body=index_body)
print(f"인덱스 '{index_name}' 생성 완료\n")

인덱스 'korean_test' 생성 완료



In [6]:
# 말뭉치 불러오기
df = pd.read_parquet('말뭉치.parquet.gzip')

In [7]:
# 표본 번호와 문장으로 그루핑
groupped = df.groupby(['표본 번호', '문장'])

In [ ]:
from tqdm import tqdm

for (i, sentence), df in tqdm(groupped, total=len(groupped), desc="말뭉치 저장 중"):
    morphs = []
    for __, row in df.iterrows():
        if row["형태 주석"] == '0':
            continue
        morphs.append(row["형태 주석"])
    
    embedding = model.encode(sentence).tolist()
    client.index(
        index=index_name,
        id=i,
        body={
            "original_text": sentence,
            "morphs": morphs,
            "embedding": embedding
        }
    )

말뭉치 저장 중:   0%|▏                                                                                                                 | 309/235902 [00:16<2:43:21, 24.04it/s]

In [14]:
# 색인 완료 대기
client.indices.refresh(index=index_name)
print("\n색인 완료!\n")


색인 완료!



In [32]:
query_text = "저는 어제 뱡완에 갔어요."

In [33]:
# 1. BM25 검색 (키워드 매칭)
print("\n[1] BM25 검색 결과:")
print("-" * 60)

bm25_query = {
    "query": {
        "match": {
            "original_text": query_text
        }
    }
}

bm25_results = client.search(index=index_name, body=bm25_query)

for hit in bm25_results['hits']['hits']:
    print(f"스코어: {hit['_score']:.4f} | {hit['_source']['original_text']}")


[1] BM25 검색 결과:
------------------------------------------------------------
스코어: 9.2223 | 휴일이었지요 어제 학교에 갔어요
스코어: 7.4715 | 저는 네팔에 보커라 갔어요.
스코어: 7.2801 | 좀 비군해서 저는 집에 갔어요.
스코어: 7.2801 | 저는 친구하고 같이 인사동에 갔어요.
스코어: 7.0696 | 친구들을 제 집에 갔어요.
스코어: 6.9259 | 특히 여름 때 저는 수영장에 친구와 같이 갔어요.
스코어: 6.6052 | 저는 작년에 내 친구들을 함께 "La Palawa"에 갔어요.
스코어: 6.5903 | 저는 홍대 있는 백화점에 갔습니다.
스코어: 6.3070 | 집에 갔어요.
스코어: 6.3070 | 학교애 갔어요.


In [34]:
# 2. 벡터 검색 (의미 유사도)
print("\n[2] 벡터 검색 결과:")
print("-" * 60)

query_embedding = model.encode(query_text).tolist()

vector_query = {
    "query": {
        "knn": {
            "embedding": {
                "vector": query_embedding,
                "k": 5
            }
        }
    }
}

vector_results = client.search(index=index_name, body=vector_query)

for hit in vector_results['hits']['hits']:
    print(f"스코어: {hit['_score']:.4f} | {hit['_source']['original_text']}")


[2] 벡터 검색 결과:
------------------------------------------------------------
스코어: 0.8460 | 페이징 상하에 지난내 갔어요
스코어: 0.8421 | 주말에 제주도 갔어요.
스코어: 0.8358 | 항쿡에 가 봤어요
스코어: 0.8355 | 해운대에 갔어요
스코어: 0.8351 | 첫날에는 저는 유니버첼스튜디오에 갔습니다.


In [35]:
# 3. 하이브리드 검색 (BM25 + 벡터)
print("\n[3] 하이브리드 검색 결과:")
print("-" * 60)

hybrid_query = {
    "query": {
        "bool": {
            "should": [
                {
                    "match": {
                        "original_text": {
                            "query": query_text,
                            "boost": 0.5  # BM25 가중치
                        }
                    }
                },
                {
                    "knn": {
                        "embedding": {
                            "vector": query_embedding,
                            "k": 5,
                            "boost": 0.5  # 벡터 가중치
                        }
                    }
                }
            ]
        }
    }
}

hybrid_results = client.search(index=index_name, body=hybrid_query)

for hit in hybrid_results['hits']['hits']:
    print(f"스코어: {hit['_score']:.4f} | {hit['_source']['original_text']}")


[3] 하이브리드 검색 결과:
------------------------------------------------------------
스코어: 4.6112 | 휴일이었지요 어제 학교에 갔어요
스코어: 3.7357 | 저는 네팔에 보커라 갔어요.
스코어: 3.6400 | 좀 비군해서 저는 집에 갔어요.
스코어: 3.6400 | 저는 친구하고 같이 인사동에 갔어요.
스코어: 3.5348 | 친구들을 제 집에 갔어요.
스코어: 3.4665 | 해운대에 갔어요
스코어: 3.4629 | 특히 여름 때 저는 수영장에 친구와 같이 갔어요.
스코어: 3.4155 | 첫날에는 저는 유니버첼스튜디오에 갔습니다.
스코어: 3.3718 | 주말에 제주도 갔어요.
스코어: 3.3026 | 저는 작년에 내 친구들을 함께 "La Palawa"에 갔어요.


# 가설 1차 실행결과
 - 형태소 분석 아예 안하고, 입력 문장을 바로 말뭉치에서 검색하는 방법 사용해봄
 - 예상대로, BM25 검색은 대체로 오타 오류를 비교적 잘 잡는 느낌.
 - 벡터 검색결과가 모호함. 의미적으로는 유사하기 때문에 비슷한 "표현" 정보는 가져올 수 있을거 같은데, 결국 다른 오류가 뜨는거 같음.

## 그러면...
 - (일단 하이브리드 방식을 사용한다 가정하고) 검색 결과에 대해 오류가 발생한 "부분" 을 다시 뽑아서 이 부분만 원본 문장과 대조해서 동일해 보이는 케이스를 역으로 검색해야 하는가?
 - 예를 들면, "저는 어제 병원에 갔어요." 라는 올바른 문장을 입력하면 아래와 같은 반환값이 옴.
```
스코어: 4.1908 | 제 건강은 안 좋아서 자주 아파요.
스코어: 3.4652 | 지금 한국에 날씨가 추워서 미나 씨 할아버지 건강이 안 좋아요.
스코어: 3.4494 | 안 좋아요.
스코어: 3.0841 | 저는 건강을 위해서 생활 안 좋은 습관을 바습니다.
스코어: 2.9577 | 하지만 미나 씨 할머니가 건강이 안 좋으셔서 간식을 안 드세요.
스코어: 2.9322 | 혼자 가니까 안 좋아 안 좋아요
스코어: 2.8821 | 하지만 술을 많이 마시면 건강에 안 좋다.
스코어: 2.8630 | 한식도 건강하고 맛있어서 좋아요.
스코어: 2.8475 | 하지만 불고기보다 김밥이 더 건강에 좋아요.
스코어: 2.6567 | 저는 건강을 위해서 안 좋은 생활 습관을 바꿀 거에요.
```
 - 이 문장들을 전부 뽑아서 형태소 단위로 N-gram 검색을 해서 오류가 존재하는지의 여부를 검사하면, 입력한 문장이 어디에 속하는지 더 잘 알 수 있지 않을까 하는 생각.
 - 